<h1>Delivery Time Prediction – Brazilian E-Commerce Public Dataset by Olist</h1>

Bu projede Brazilian E-Commerce Public Dataset by Olist kullanılarak müşteri siparişlerinin teslimat süreleri analiz edilmektedir. Veri ön işleme, keşifsel veri analizi (EDA) ve özellik mühendisliği adımlarının ardından, Linear Regression, Random Forest ve XGBoost modelleri ile tahmini teslim süresi (gün cinsinden) hesaplanmış ve modellerin performansları karşılaştırılmıştır.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_palette("Set2")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import string
import random
import re

import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb


In [ ]:
pd.set_option("display.max_columns" , None)
plt.style.use("seaborn-v0_8")
sns.set_palette("Set2") #pastel tonlar

In [ ]:
# Yardımcı: eksik değer tablosu
def missing_table(df):
    mis = df.isna().sum()
    pct = (mis / len(df) * 100).round(2)
    out = pd.DataFrame({"missing": mis, "%": pct})
    return out[out["missing"] > 0].sort_values("missing", ascending=False)

In [ ]:
orders = pd.read_csv("olist_orders_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
cat_trans = pd.read_csv("product_category_name_translation.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
geo = pd.read_csv("olist_geolocation_dataset.csv")

In [ ]:
print("Orders shape:", orders.shape)
print("Customers shape:", customers.shape)
print("Products shape:", products.shape)
print("Sellers shape:", sellers.shape)
print("Order Items shape:", order_items.shape)
print("Payments shape:", payments.shape)
print("Reviews shape:", reviews.shape)
print("Geo shape:", geo.shape)

In [ ]:
display(orders.head(3))
orders.info()
missing_table(orders)

In [ ]:
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
for c in date_cols:
    orders[c] = pd.to_datetime(orders[c], errors="coerce")

reviews["review_creation_date"]  = pd.to_datetime(reviews["review_creation_date"],  errors="coerce")
reviews["review_answer_timestamp"]= pd.to_datetime(reviews["review_answer_timestamp"], errors="coerce")


<h3>1. Customers</h3>

In [ ]:
total = len(customers)
ax1 = plt.figure(figsize=(17,7))

g = sns.countplot(x='customer_state', data=customers, palette = 'gnuplot')
g.set_title("Eyaletlere Göre Müşteri Sayısı / Customers by States", fontsize=20)
g.set_xlabel("Eyaletler / States", fontsize=17)
g.set_ylabel("Müşteri Sayısı / Count", fontsize=17)
sizes = []
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=10)
g.set_ylim(0, max(sizes) * 1.1)

In [ ]:
plt.figure(figsize=(15,6))
top_10_cust_states = customers.groupby('customer_city')['customer_id'].count() \
.sort_values(ascending = False).head(10)
sns.barplot(x=top_10_cust_states.index, y = top_10_cust_states.values, palette = 'gnuplot')
plt.title('Top 10 Cities')
plt.show()

In [ ]:
# shipping_limit_date sütununu datetime tipine dönüştür
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])

# Yıl, Ay, Gün ayrı sütunlar
order_items['Years'] = order_items['shipping_limit_date'].dt.year
order_items['Month'] = order_items['shipping_limit_date'].dt.month
order_items['Days']  = order_items['shipping_limit_date'].dt.day



In [ ]:
order_items['Years'] = order_items['shipping_limit_date'].dt.year
order_items['Month'], order_items['Days'] = (order_items['shipping_limit_date'].dt.month, order_items['shipping_limit_date'].dt.day)
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date']).dt.date

<h3>2. Order Items</h3>

In [ ]:
pl1 = plt.figure(figsize=(15,6))
month = order_items.groupby('Days')['product_id'].count().sort_values(ascending=False).head(112650)
sns.barplot(x=month.index, y=month.values, palette = 'gnuplot')
pl1 = pl1 = plt.xticks(rotation=40)
pl1 = plt.xlabel('Günler / Days')
pl1 = plt.title('Günlere Göre Satışlar/ Sales Days');
pl1 = plt.show()

In [ ]:
pl2 = plt.figure(figsize=(15,6))
pl2 = year = order_items.groupby('Month')['product_id'].count().sort_values(ascending=False).head(112650)
pl2 = sns.barplot(x=year.index, y=year.values, palette = 'gnuplot')
pl2 = plt.xticks(rotation=40)
pl2 = plt.xlabel('Aylar / Months')
pl2 = plt.title('Aylara Göre Satışlar / Sale in the Month');
pl2 = plt.show()

In [ ]:
pl3 = plt.figure(figsize=(10,6))
pl3 = year = order_items.groupby('Years')['product_id'].count().sort_values(ascending=False).head(112650)
pl3 = sns.barplot(x=year.index, y=year.values, palette = 'gnuplot')
pl3 = plt.xticks(rotation=30)
pl3 = plt.xlabel('Yıl / Year')
pl3 = plt.title('Yıllara Göre Satışlar / Sales Years');
pl3 = plt.show()

<h3>3. Order Payment</h3

In [ ]:
total = len(payments)

# Stil ve figür ayır
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(16,5))

# Countplot çiz
g = sns.countplot(x='payment_installments', data=payments, palette='gnuplot', ax=ax)

# Başlık ve label ekle
g.set_title("Taksitler / Installments", fontsize=20)
g.set_xlabel("Taksitler / Installments", fontsize=17)
g.set_ylabel("Quantidade / Count", fontsize=17)
# Bar üstüne yüzde yaz
sizes = []
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x() + p.get_width()/2.,
           height + 3,
           '{:1.2f}%'.format(height/total*100),
           ha="center", fontsize=11)
# Y ekseni ayarı
g.set_ylim(0, max(sizes) * 1.1)
plt.show()


<h3>4. Sellers</h3>

In [ ]:
total = len(sellers)
ax8 = plt.figure(figsize=(20,7))
ax8 = plt.style.use('dark_background')
g = sns.countplot(x='seller_state', data=sellers, palette = 'gnuplot')
g.set_title("Satıcıların Dağılımı / Sellers Distribution", fontsize=20)
g.set_xlabel("Eyaletler / States", fontsize=17)
g.set_ylabel("Sayı / Count", fontsize=17)
sizes = []
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=10)
g.set_ylim(0, max(sizes) * 1.1)

<h3>Kategori Çeviri</h3>

In [ ]:
products_en = products.merge(
    cat_trans,
    on="product_category_name",   # orijinal isim üzerinden eşleştir
    how="left"
)
products_en = products_en.drop(columns=["product_category_name"]) \
                         .rename(columns={"product_category_name_english": "product_category_name"})
products_en.head()

In [ ]:
plt.figure(figsize=(20,6))
top_25_prod_categories = products_en.groupby('product_category_name')['product_id'].count().sort_values(ascending=False).head(25)
sns.barplot(x=top_25_prod_categories.index, y=top_25_prod_categories.values, palette = 'gnuplot')
plt.xticks(rotation=70)
plt.xlabel('Ürün Kategorileri / Product Category')
plt.title(' En Yaygın 25 Kategori / Top 25 Most Common Categories');
plt.show()

<h2>Predicting Order Delivery Days using Linear, RF, and XGBoost Models</h2>

# Veri Hazırlık

In [ ]:
# 1) order_items özetleri
items_agg = (order_items
             .groupby("order_id")
             .agg(n_items=("order_item_id","count"),
                  products_total_price=("price","sum"),
                  freight_total=("freight_value","sum"))
             .reset_index())

# 2) payments özetleri (taksit + en sık ödeme tipi)
pay_agg = (payments
           .groupby("order_id")
           .agg(installments_max=("payment_installments","max"),
                payment_types=("payment_type", lambda s: s.value_counts().index[0]))
           .reset_index())

# 3) siparişin baskın satıcısı (mode) → seller_state
seller_state_per_order = (order_items[["order_id","seller_id"]]
    .merge(sellers[["seller_id","seller_state"]], on="seller_id", how="left")
    .groupby("order_id")["seller_state"]
    .agg(lambda x: x.mode().iloc[0] if len(x)>0 else np.nan)
    .reset_index())

# 4) baskın kategori (en pahalı kalem)
dom_cat = (order_items
           .merge(products_en[["product_id","product_category_name"]], on="product_id", how="left")
           .sort_values(["order_id","price"], ascending=[True, False])
           .drop_duplicates("order_id")[["order_id","product_category_name"]])

# 5) orders + customers
ord_cust = orders.merge(customers[["customer_id","customer_state"]], on="customer_id", how="left")

# 6) Hepsini birleştir → df_
df_ = (ord_cust
       .merge(items_agg, on="order_id", how="left")
       .merge(pay_agg, on="order_id", how="left")
       .merge(seller_state_per_order, on="order_id", how="left")
       .merge(dom_cat, on="order_id", how="left")
       .copy())

# 7) Zaman alanları ve hedef
df_["delivery_days_actual"] = (df_["order_delivered_customer_date"] - df_["order_purchase_timestamp"]).dt.days
df_["est_days"]             = (df_["order_estimated_delivery_date"] - df_["order_purchase_timestamp"]).dt.days

# 8) Sepet değeri + log
df_["order_value"] = df_["products_total_price"].fillna(0) + df_["freight_total"].fillna(0)
q99 = df_["order_value"].quantile(0.99)
df_["order_value_capped"] = np.where(df_["order_value"] > q99, q99, df_["order_value"])
df_["order_value_log"] = np.log1p(df_["order_value_capped"])

# 9) Zaman türevleri
df_["purchase_month"] = df_["order_purchase_timestamp"].dt.month
df_["purchase_dow"]   = df_["order_purchase_timestamp"].dt.dayofweek
df_["purchase_hour"]  = df_["order_purchase_timestamp"].dt.hour

# 10) Eksik hedefleri at + hedefi yumuşat
df_ = df_.dropna(subset=["order_delivered_customer_date","order_purchase_timestamp","order_estimated_delivery_date"]).copy()
lo, hi = df_["delivery_days_actual"].quantile([0.01, 0.99])
df_["delivery_days_actual_c"] = df_["delivery_days_actual"].clip(lo, hi)

# 11) Eksikleri doldur / nadir kategorileri toparla (hız için)
def collapse_rare(s, min_count=100, other="other"):
    if s.dtype != "object":
        return s
    vc = s.value_counts(dropna=False)
    rare = vc[vc < min_count].index
    return s.where(~s.isin(rare), other)

for col in ["payment_types","customer_state","seller_state","product_category_name"]:
    if col in df_.columns:
        df_[col] = df_[col].fillna("other")
        df_[col] = collapse_rare(df_[col], min_count=100)

# 12) Özellik seti
features = [
    "est_days","n_items","order_value_log","installments_max","payment_types",
    "customer_state","seller_state","product_category_name",
    "purchase_month","purchase_dow","purchase_hour"
]

In [ ]:
# 1) Sapma: actual - est  (pozitif = geç teslim)
df_ = df_.copy()
df_["delay_residual"] = df_["delivery_days_actual"] - df_["est_days"]

# Sadece görselleştirme için uç kırpma
r_lo, r_hi = df_["delay_residual"].quantile([0.01, 0.99])
df_["delay_residual_c"] = df_["delay_residual"].clip(r_lo, r_hi)

# 2) Zaman türevleri (ay periyodu ve hafta sonu bayrağı)
ts = df_["order_purchase_timestamp"]
df_["month_ts"]   = ts.dt.to_period("M").dt.to_timestamp()
df_["is_weekend"] = ts.dt.dayofweek.isin([5,6]).astype(int)

# 3) n_items bucket (okunurluk için)
df_["n_items_bucket"] = pd.cut(df_["n_items"].fillna(0),
                               bins=[-0.1,1,2,4,7,1e9],
                               labels=["1","2","3-4","5-7","8+"])
# 4) Kategori top-N (okunurluk için)
def topk_or_other(s, k=12, other="other"):
    top = s.value_counts().nlargest(k).index
    return s.where(s.isin(top), other)
df_["product_category_top"] = topk_or_other(df_["product_category_name"], k=12)


# Son kontrol – eksik varsa burada görürsün
missing = [c for c in features if c not in df_.columns]
print("Eksik kolon yok " if not missing else f"Eksik kolon(lar): {missing}")


<h3> 1) Teslim ve Sapma Dağılımı </h3>

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.histplot(df_["delivery_days_actual_c"], bins=40, kde=True)
plt.title("Teslim Süresi Dağılımı (gün)")
plt.xlabel("delivery_days_actual (clip'li)")

plt.subplot(1,2,2)
sns.histplot(df_["delay_residual_c"], bins=40, kde=True)
plt.axvline(0, color="red", ls="--", label="0 (tahminle eşit)")
plt.title("Tahmine Göre Sapma Dağılımı (actual - est)")
plt.xlabel("delay_residual (clip'li)")
plt.legend()
plt.tight_layout(); plt.show()

<h4>Teslim Süresi Dağılımı – gün</h4>
Ortalama teslim süresinin 6, 7–12 gün arasında yoğunlaştığını gözlemliyoruz. 20 günün üzerindeki teslimatlar ise nadir ama mevcut, yani bazı siparişlerde ciddi gecikmeler yaşanabiliyor. Bu dağılım bize lojistik süreçlerin genelde planlanan zaman aralığında olduğunu ama uzun kuyruklu gecikmelerin de olduğunu gösteriyor.
<h4>Tahmine Göre Sapma Dağılımı – actual - est</h4>
Çoğu siparişin -10 ,-8 ile -20 gün arası sapma gösterdiğini görüyoruz, yani tahminler sistematik olarak gerçekleşenden biraz daha uzun süre öngörmüş. Yani model genelde güvenli tarafta kalıyor ama bazen fazla temkinli davranıyor.

Teslimatlar genellikle 10 gün civarında gerçekleşiyor, model ise teslim sürelerini biraz daha uzun öngörerek genelde güvenli tarafta kalıyor.

<h3>3) Aylık Trend: actual & sapma</h3>


In [ ]:
monthly = (df_.groupby("month_ts")[["delivery_days_actual","delay_residual"]]
           .mean().reset_index())
plt.figure(figsize=(10,4))
sns.lineplot(data=monthly, x="month_ts", y="delivery_days_actual", label="Ortalama actual (gün)")
sns.lineplot(data=monthly, x="month_ts", y="delay_residual", label="Ortalama sapma (gün)")
plt.title("Aylık Ortalama Teslim Süresi ve Sapma")
plt.xlabel("Ay"); plt.ylabel("Gün"); plt.legend()
plt.tight_layout(); plt.show()

Başlangıçta, yani 2016’nın son aylarında teslim süreleri çok yüksek ama zamanla hızlı bir düşüş yaşanmış. 2017’den itibaren ortalama teslim süresi daha dengeli bir seviyeye oturmuş, genellikle 10–15 gün arasında kalmış.
<br>Teslim süreleri zamanla istikrarlı hale geldi, model ise genelde gerçek süreden daha uzun tahmin ederek temkinli kaldı.

<h3>5) Ürün Adedi (bucket) – Ortalama Teslim</h3>

In [ ]:
plt.figure(figsize=(6,4))
order = ["1","2","3-4","5-7","8+"]
sns.barplot(data=df_, x="n_items_bucket", y="delivery_days_actual_c",
            order=order, estimator=np.mean, ci=None)
plt.title("Ürün Adedi (bucket) vs Ortalama Teslim (gün)")
plt.xlabel("n_items bucket"); plt.ylabel("Ortalama gün")
plt.tight_layout(); plt.show()

Ürün sayısı teslim süresini büyük ölçüde etkilemiyor, yalnızca çoklu siparişlerde (8+) teslim süresi biraz uzuyor.

<h3>6) Kategori Bazında Teslim Süresi (Top 12)</h3>

In [ ]:
plt.figure(figsize=(11,6))
sns.boxplot(data=df_[df_["product_category_top"].notna()],
            y="product_category_top", x="delivery_days_actual_c", showfliers=False)
plt.title("Kategori Bazında Teslim Süresi (Top 12, outlier gizli)")
plt.xlabel("Teslim (gün)"); plt.ylabel("Kategori")
plt.tight_layout(); plt.show()

Genel olarak kategoriler arasında teslim süresi 10–15 gün aralığında yoğunlaşıyor. En hızlı teslimatlar *other* ve *housewares* gibi kategoriler , en uzun teslim süresi *telephony* kategorisinde.

Diğer kategorilerde ise teslim süreleri birbirine yakın ve büyük farklılık göstermiyor.

<h3>7) Koridor Isı Haritası (müşteri × satıcı – ort. sapma)</h3>



In [ ]:
# en sık 8 müşteri/8 satıcı eyaleti
top_c = df_["customer_state"].value_counts().nlargest(8).index
top_s = df_["seller_state"].value_counts().nlargest(8).index
corr_mat = (df_[df_["customer_state"].isin(top_c) & df_["seller_state"].isin(top_s)]
            .pivot_table(index="customer_state", columns="seller_state",
                         values="delay_residual", aggfunc="mean"))
plt.figure(figsize=(8,5))
sns.heatmap(corr_mat, annot=True, fmt=".1f", cmap="coolwarm", center=0)
plt.title("Koridor (customer × seller) Ortalama Sapma (gün)")
plt.xlabel("Seller state"); plt.ylabel("Customer state")
plt.tight_layout(); plt.show()

Çoğu eyalet hattında teslimatlar tahminden daha erken gerçekleşiyor, en belirgin sapma MG ve RS arasında.

# Modeller

In [ ]:
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

# X / y
X = df_[features].copy()
y = df_["delivery_days_actual_c"].astype(float).copy()

cat_cols = X.select_dtypes(include=["object","category"]).columns.tolist()
num_cols = [c for c in features if c not in cat_cols]

pre = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc", StandardScaler())]), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("ohe", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
])

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

models = [
    ("Linear", Pipeline([("pre", pre), ("m", LinearRegression())])),
    ("RF",     Pipeline([("pre", pre), ("m", RandomForestRegressor(
        n_estimators=150, max_depth=10, min_samples_leaf=3,
        n_jobs=-1, random_state=42))]))
]
if HAS_XGB:
    models.append(("XGB", Pipeline([("pre", pre), ("m", XGBRegressor(
        n_estimators=300, max_depth=6, learning_rate=0.10,
        subsample=0.9, colsample_bytree=0.8, tree_method="hist",
        n_jobs=-1, random_state=42))])))

# Eğitim ve değerlendirme
rows = []
best_name, best_pipe, best_rmse = None, None, 1e9
for name, pipe in models:
    pipe.fit(X_tr, y_tr)
    pred = pipe.predict(X_te)
    mse  = mean_squared_error(y_te, pred); rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_te, pred); r2 = r2_score(y_te, pred)
    rows.append({"Model": name, "RMSE": rmse, "MAE": mae, "R2": r2})
    if rmse < best_rmse:
        best_name, best_pipe, best_rmse = name, pipe, rmse

res = pd.DataFrame(rows).sort_values("RMSE")
print(res)
print(f"\nSeçilen en iyi model: {best_name} (RMSE={best_rmse:.2f})")



XGBoost en iyi sonuç verdi (en düşük RMSE, en yüksek R²).

In [ ]:

# res DataFrame'i long formata çevir
res_melt = res.melt(id_vars="Model", value_vars=["RMSE","MAE","R2"],
                    var_name="Metrik", value_name="Değer")

plt.figure(figsize=(9,5))
sns.barplot(data=res_melt, x="Model", y="Değer", hue="Metrik", palette="Set2")

# değerleri yazdır
for p in plt.gca().patches:
    plt.gca().text(p.get_x() + p.get_width()/2,
                   p.get_height() + 0.02,
                   f"{p.get_height():.2f}",
                   ha="center", va="bottom", fontsize=9)

plt.title("LinearRegression, RandomForest ve XGBoost Karşılaştırması")
plt.ylabel("Skor")
plt.xlabel("Model")
plt.legend(title="Metrik")
plt.tight_layout()
plt.show()


- RMSE ve MAE değerleri en düşük XGBoost modelinde. Bu, tahmin hatalarının diğer modellere göre daha az olduğunu gösteriyor.
- R² skoru da XGBoost’ta en yüksek (0.39). Yani XGBoost, veriyi diğer modellere kıyasla daha iyi açıklıyor.
- RandomForest XGBoost’a yakın bir performans sergiliyor ama biraz daha zayıf.
- Lineer Regresyon ise en düşük başarıyı gösteriyor; hem hata değerleri yüksek hem de R² en düşük.

<h3>1) Sapma dağılımı (bizim model vs gerçek)</h3>

In [ ]:
residuals = y_te - pred
plt.figure(figsize=(10,4))
sns.histplot(residuals, bins=40, kde=True)
plt.axvline(0, color="red", ls="--", label="0 hata çizgisi")
plt.title(f"{best_name} Modeli - Hata (Gerçek - Tahmin)")
plt.xlabel("Sapma (gün)")
plt.ylabel("Frekans")
plt.legend(); plt.tight_layout(); plt.show()

XGBoost hataları sıfıra yakın ve dengeli, ancak genelde teslimatı olduğundan uzun tahmin ediyor

<h3>2) Gerçek vs Tahmin – Scatter</h3>

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_te, pred, alpha=0.3)
m, M = y_te.min(), y_te.max()
plt.plot([m, M], [m, M], "r--", label="Mükemmel tahmin")
plt.xlabel("Gerçek Teslim Süresi (gün)")
plt.ylabel("Tahmini Teslim Süresi (gün)")
plt.title(f"Gerçek vs Tahmin ({best_name})")
plt.legend(); plt.tight_layout(); plt.show()

<h3>3) Aylık trend: ortalama gerçek vs bizim model</h3>

In [ ]:
tmp = pd.DataFrame({"y_te": y_te, "y_pred": pred})
tmp["month"] = df_.loc[y_te.index, "purchase_month"]

monthly_cmp = tmp.groupby("month")[["y_te","y_pred"]].mean().reset_index()

plt.figure(figsize=(9,4))
sns.lineplot(data=monthly_cmp, x="month", y="y_te", label="Gerçek ort. teslim")
sns.lineplot(data=monthly_cmp, x="month", y="y_pred", label=f"{best_name} tahmin")
plt.title("Aylık Ortalama Teslim Süresi – Gerçek vs Model")
plt.xlabel("Ay"); plt.ylabel("Ortalama Gün")
plt.legend(); plt.tight_layout(); plt.show()

Model, aylık teslim sürelerini büyük ölçüde doğru tahmin ediyor, sadece yılın başı ve sonunda küçük sapmalar var.

<h3>4) Kategori Bazında Ortalama Teslim Süresi – Gerçek vs Model</h3>

In [ ]:
tmp = pd.DataFrame({"y_te": y_te, "y_pred": pred}, index=y_te.index)
tmp["cat"] = df_.loc[y_te.index, "product_category_name"]

# en popüler 10 kategori
cat_cmp = (tmp.groupby("cat")[["y_te","y_pred"]]
           .mean()
           .join(tmp["cat"].value_counts().rename("n"))
           .sort_values("n", ascending=False)
           .head(10)
           .drop(columns="n")
           .reset_index())

x = np.arange(len(cat_cmp))
w = 0.38

plt.figure(figsize=(12,5))
plt.bar(x - w/2, cat_cmp["y_te"],  width=w, label="Gerçek")
plt.bar(x + w/2, cat_cmp["y_pred"], width=w, label="Tahmin", alpha=0.85)

plt.xticks(x, cat_cmp["cat"], rotation=30, ha="right")
plt.ylabel("Gün")
plt.title("Kategori Bazında Ortalama Teslim Süresi – Gerçek vs Model")
plt.legend()
plt.tight_layout()
plt.show()
